Creating a custom CNN for detection of cancer from histology specimens.

##first imports for torch, torchvision, numpy, pandas, and os

In [1]:
import torch
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import PIL
import matplotlib as plt

In [2]:
import os
import pandas as pd
import numpy as np
import shutil

In [ ]:
#Set up for cuda usage
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
print("{} is available".format(device))

In [6]:
# Reorganizing data to subfolders zero and one for pytorch labels implementation
csv_file = r'''C:\Users\gtm12\OneDrive\Documents\Jupyter Notebooks\Kaggle\Histo_cancer\train_labels.csv'''
training_path = r'''C:\Users\gtm12\OneDrive\Documents\Jupyter Notebooks\Kaggle\Histo_cancer\histo_data\train\\'''
train_path_0 = r'''C:\Users\gtm12\OneDrive\Documents\Jupyter Notebooks\Kaggle\Histo_cancer\histo_data\train\0\\'''
train_path_1 = r'''C:\Users\gtm12\OneDrive\Documents\Jupyter Notebooks\Kaggle\Histo_cancer\histo_data\train\1\\'''

def move_to_subfolders(csv_file, training_path, train_path_0, train_path_1):
    labels = pd.read_csv(csv_file)


    try:
        for i in range(len(labels)):
            file_name = str(labels.id[i]) + '.tif'
            if labels.label[i].item() == 0:
                file_path_copy = train_path_0
            elif labels.label[i].item() == 1:
                file_path_copy = train_path_1

            file_path_copy = file_path_copy + file_name
            file_origin = training_path + file_name
            shutil.copy2(file_origin, file_path_copy)
    
            os.remove(file_origin)
    except FileNotFoundError:
        print("There are no filese or files have been moved")
    return

In [7]:
move_to_subfolders(csv_file, training_path, train_path_0, train_path_1)

There are no filese or files have been moved


In [ ]:
# Begin to load Data Transforms and split validation set

In [ ]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 1000
# percentage of training set to use as validation
valid_size = 0.15
#file directories
data_dir = '/content/drive/My Drive/pytorch final/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'

In [ ]:
# TODO: Define your transforms for the training and validation sets
train_transforms = transforms.Compose([transforms.RandomRotation(15),
                                      transforms.RandomResizedCrop(32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ColorJitter(brightness=0.4,
                                                             contrast=0.4,
                                                             saturation = 0.4,
                                                             hue=0.1),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])

valid_transforms = transforms.Compose([transforms.Resize(48),
                                      transforms.CenterCrop(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])


# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True)